___
# Insper - Ciência dos Dados - Projeto 1

# Obtenção das notícias

Utilize este notebook para **baixar notícias** sobre algum assunto de seu interesse.

As notícias serão automaticamente baixadas do site https://www.moneytimes.com.br/ utilizando o termo de busca informado por você!

Será gerado um arquivo `dados.xlsx` contendo as informações disponíveis para o projeto.

## Preparação do ambiente no jupyter

Vamos importar algumas bibliotecas e definir algumas funções úteis!

In [14]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import os.path
from math import floor

from ipywidgets import IntProgress
from IPython.display import display

In [15]:
def parse_primeiro_paragrafo(content):
    
    soup = BeautifulSoup(content, 'html.parser')    
    single__text = soup.find('div', class_='single__text')
    prim_parag = single__text.find('p').text
    
    return prim_parag
    
    
def parse_result(content, page=1):
    
    soup = BeautifulSoup(content, 'html.parser')

    lista_tag_noticia = soup.find_all('div', class_='news-item')

    lista_categoria = []
    lista_titulo = []
    lista_prim_parag = []
    lista_data = []

    for i in range(0, len(lista_tag_noticia)-1):
        tag_noticia = lista_tag_noticia[i]

        categoria = tag_noticia.find('div', class_='news-item__category').text
        categoria = categoria.replace('\n', '')
        categoria = categoria.replace('\t', '')
        lista_categoria.append(categoria)

        tag_titulo = tag_noticia.find('h2', class_='news-item__title')
        titulo = tag_titulo.text
        lista_titulo.append(titulo)
        
        link_noticia = tag_titulo.find('a')['href']
        content_noticia = get_content(link_noticia)
        prim_parag = parse_primeiro_paragrafo(content_noticia)
        lista_prim_parag.append(prim_parag)

        data_hora = tag_noticia.find('span', class_='date').text
        lista_data.append(data_hora)

    df = pd.DataFrame({'Categoria': lista_categoria,
                       'Titulo': lista_titulo,
                       'PrimeiroParag': lista_prim_parag,
                       'Data': lista_data,
                       'Pagina': page,
                       'Target': None
                  })
    return df
    
def get_content(url):
    headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    
    resposta = requests.get(url=url, headers=headers)
    resposta.encoding = 'utf-8'
    
    return resposta.text

def get_news_page(word, page):
    url = f'https://www.moneytimes.com.br/page/{page}/?s={word}'
    content = get_content(url)
    df_res = parse_result(content, page)
    return df_res

def get_df_news(word, n_news=500):
    page = 0
    news_per_page = 10
    
    print('\nProgresso:')
    n_max_pages = n_news//news_per_page
    prog_bar = IntProgress(min=0, max=n_max_pages)
    display(prog_bar)


    while True:
        page = page + 1
        df_aux = get_news_page(word, page)
        
        if page==1:
            df = df_aux
        else:
            df = pd.concat([df, df_aux], ignore_index=True)
        
        prog_bar.value += 1
        
        if page==n_max_pages:
            break
            
    return df

def get_news(word, n_news=500, perc_train=0.6):
    
    if not os.path.isfile('./dados1.xlsx'):
        
        print(f'Ok! Vou baixar notícias sobre o termo {word} no MoneyTimes.com.br!')
        print('\nAguarde... Este processo pode demorar alguns minutos!')
        
        df = get_df_news(word, n_news)
        df = df.sample(frac=1.0)
    
        writer = pd.ExcelWriter('./dados.xlsx')
        
        n_real = len(df)
        n_treino = floor(n_real * perc_train)

        dft = df.iloc[:n_treino]
        dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

        dfc = df.iloc[n_treino:]
        dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

        writer.save()
        
        print('Processo finalizado!')
        print('\nGeramos um arquivo chamado dados.xlsx, confira!')
        
    else:
        
        raise Exception('Arquivo dados.xlsx já existe! Apague o arquivo caso queira gerar novamente!')


## Defina o seu assunto

Execute a próxima célula e defina um assunto sobre o qual gostaria de encontrar notícias para o Projeto 1.

Exemplos:
- soja
- nft
- ministro
- dólar

In [16]:
assunto_noticia = input('Quer pesquisar notícias de qual assunto?! ')

## Defina a quantidade de notícias

Execute a próxima célula e escolha a quantidade de notícias a serem baixadas. A quantidade final retornada pode ser inferior ao desejado caso o seu assunto não seja muito abordado pelo MoneyTimes. Se isto ocorrer, escolha outro assunto.

In [17]:
qtde_noticia = int(input('Qual a quantidade desejada de notícias?! '))

ValueError: invalid literal for int() with base 10: ''

## Obtenção das notícias

Execute a próxima célula. Ela irá construir uma base de dados de notícias sobre o seu assunto de interesse!

Este processo poderá demorar alguns minutos. Quando ele finalizar, será gerado um arquivo `dados.xlsx` com as notícias!

Pontos de atenção:
- Confira se a quantidade de notícias encontradas foi suficiente (conforme enunciado)

In [ ]:
get_news(assunto_noticia, n_news=qtde_noticia, perc_train=0.6)

Ok! Vou baixar notícias sobre o termo  no MoneyTimes.com.br!

Aguarde... Este processo pode demorar alguns minutos!

Progresso:


IntProgress(value=0, max=50)

KeyboardInterrupt: 